In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import cebra_v2 as cebra2
from collections import defaultdict
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA,FastICA
from torch.utils.data.sampler import BatchSampler
import package
import pandas as pd
import matplotlib.animation as animation
import tembedding
import scipy as sc
from skimage.metrics import structural_similarity as ssim
from sklearn.cluster import KMeans,OPTICS
import math
from joblib import Memory,Parallel,delayed,parallel_backend
import time
from multiprocessing import Lock, Process, Queue
import multiprocessing
import queue 
import os
import networkx as nx
from scipy.spatial.transform import Rotation as R

In [2]:
dfc = np.load('/neurospin/lbi/monkeyfmri/deepstim/database/ANESTHETIC_database/derivatives/reference_kmeans/inputs/inputs.npy')
meta = pd.read_csv("/neurospin/lbi/monkeyfmri/deepstim/database/ANESTHETIC_database/derivatives/reference_kmeans/inputs/metadata.tsv", sep="\t")

In [3]:
n_runs = len(set(meta["unique_id"] + meta["monkey"]))
n_wins = 464
dfc_train = dfc[meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])].reshape((-1, n_wins, 82, 82))
dfc_test = dfc[meta["monkey"] == "jade"].reshape((-1, n_wins, 82, 82))
dfc_all = dfc.reshape((-1, n_wins, 82, 82))
dfc_all_tensor = torch.from_numpy(dfc_all).type(torch.float32)
dfc_all_tensor_norm = dfc_all_tensor - torch.mean(dfc_all_tensor,dim = [-2,-1]).reshape(156,464,1,1)

In [7]:
import importlib
importlib.reload(package.preprocessing)

def STRUCTURE(x,y):
    return 1 - package.preprocessing.structure_batch(x,y)

## ALL

In [4]:
distance = np.load("/volatile/aurelien_stumpf_mascles/project/data/DistanceMatrices/all/distance_all.npy")

In [8]:
session = 0
for i in range(1,156):
    print("Session {} : ok".format(i))
    for j in range(464):
        accu = STRUCTURE(dfc_all_tensor_norm[session,j,:,:],dfc_all_tensor_norm[i,:,:,:]).detach().numpy()
        distance[session,j,i,:] = accu
        distance[i,:,session,j] = accu

Session 1 : ok
Session 2 : ok


KeyboardInterrupt: 

: 

In [17]:
np.save("/volatile/aurelien_stumpf_mascles/project/data/DistanceMatrices/all/distance_all.npy",arr = distance)

In [18]:
for i in range(156):
    print("Session {} : ok".format(i))
    for j in range(464):
        accu = STRUCTURE(dfc_all_tensor_norm[0,j,:,:],dfc_all_tensor_norm[i,:,:,:]).detach().numpy()
        distance[1,j,i,:] = accu
        distance[i,:,1,j] = accu

Session 0 : ok
Session 1 : ok
Session 2 : ok
Session 3 : ok
Session 4 : ok
Session 5 : ok
Session 6 : ok
Session 7 : ok
Session 8 : ok
Session 9 : ok
Session 10 : ok
Session 11 : ok
Session 12 : ok
Session 13 : ok
Session 14 : ok
Session 15 : ok
Session 16 : ok
Session 17 : ok
Session 18 : ok
Session 19 : ok
Session 20 : ok
Session 21 : ok
Session 22 : ok
Session 23 : ok
Session 24 : ok
Session 25 : ok
Session 26 : ok
Session 27 : ok
Session 28 : ok
Session 29 : ok
Session 30 : ok
Session 31 : ok
Session 32 : ok
Session 33 : ok
Session 34 : ok
Session 35 : ok
Session 36 : ok
Session 37 : ok
Session 38 : ok
Session 39 : ok
Session 40 : ok
Session 41 : ok
Session 42 : ok
Session 43 : ok
Session 44 : ok
Session 45 : ok
Session 46 : ok
Session 47 : ok
Session 48 : ok
Session 49 : ok
Session 50 : ok
Session 51 : ok
Session 52 : ok
Session 53 : ok
Session 54 : ok
Session 55 : ok
Session 56 : ok
Session 57 : ok
Session 58 : ok
Session 59 : ok
Session 60 : ok
Session 61 : ok
Session 62 : ok
Se

In [19]:
np.save("/volatile/aurelien_stumpf_mascles/project/data/DistanceMatrices/all/distance_all.npy",arr = distance)

## Deep-Sevoflurane

In [ ]:
dfc_train_deep_sevoflurane = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "deep-sevoflurane")].reshape(-1,n_wins,82,82)
distance_matrix = np.zeros((1,464,1,464)) 
fc_train = torch.from_numpy(dfc_train_deep_sevoflurane[0:1,:,:,:].reshape((1,464,1,82,82)))
real_size = min(11, 82, 82) # window should be atleast 11x11 

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

generator = generator_matrix_distance(fc_train)
parallel = Parallel(n_jobs=3,require='sharedmem',max_nbytes=1e5)
parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)

In [7]:
n=8
dfc_train_deep_sevoflurane = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "deep-sevoflurane")].reshape(-1,n_wins,82,82)

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

for i in range(7,n):
    print("Début de la session {}".format(i))
    distance_matrix = np.zeros((i+1,464,i+1,464)) 
    distance = np.load("./DistanceMatrices/deep_sevoflurane/distance_matrix_{}_sessions_structure.npy".format(i))
    fc_train = torch.from_numpy(dfc_train_deep_sevoflurane[0:i+1,:,:,:].reshape((i+1,464,1,82,82)))
    distance_matrix[:i,:,:i,:] = distance
    generator = generator_matrix_distance_follow(fc_train)
    parallel = Parallel(n_jobs=4,require='sharedmem',max_nbytes=1e5)
    parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
    np.save(file = "./DistanceMatrices/deep_sevoflurane/distance_matrix_{}_sessions_structure.npy".format(i+1),arr = distance_matrix)

Début de la session 7
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

## Awake

In [6]:
n=7
dfc_train_awake = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "awake")].reshape(-1,n_wins,82,82)

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

for i in range(6,n):
    print("Début de la session {}".format(i))
    distance_matrix = np.zeros((i+1,464,i+1,464)) 
    distance = np.load("./DistanceMatrices/awake/distance_matrix_{}_sessions_structure.npy".format(i))
    fc_train = torch.from_numpy(dfc_train_awake[0:i+1,:,:,:].reshape((i+1,464,1,82,82)))
    distance_matrix[:i,:,:i,:] = distance
    generator = generator_matrix_distance_follow(fc_train)
    parallel = Parallel(n_jobs=4,require='sharedmem',max_nbytes=1e5)
    parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
    np.save(file = "./DistanceMatrices/awake/distance_matrix_{}_sessions_structure.npy".format(i+1),arr = distance_matrix)

Début de la session 6
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

# Deep-propofol

In [8]:
dfc_train_deep_propofol = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "deep-propofol")].reshape(-1,n_wins,82,82)
distance_matrix = np.zeros((1,464,1,464)) 
fc_train = torch.from_numpy(dfc_train_deep_propofol[0:1,:,:,:].reshape((1,464,1,82,82)))
real_size = min(11, 82, 82) # window should be atleast 11x11 

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

generator = generator_matrix_distance(fc_train)
parallel = Parallel(n_jobs=3,require='sharedmem',max_nbytes=1e5)
parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
np.save(file = "./DistanceMatrices/deep-propofol/distance_matrix_1_sessions_structure.npy",arr = distance_matrix)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


FileNotFoundError: [Errno 2] No such file or directory: './DistanceMatrices/deep-propofol/distance_matrix_1_sessions_structure.npy'

In [11]:
n=7
dfc_train_deep_propofol = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "deep-propofol")].reshape(-1,n_wins,82,82)

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

for i in range(1,n):
    print("Début de la session {}".format(i))
    distance_matrix = np.zeros((i+1,464,i+1,464)) 
    distance = np.load("./DistanceMatrices/deep_propofol/distance_matrix_{}_sessions_structure.npy".format(i))
    fc_train = torch.from_numpy(dfc_train_deep_propofol[0:i+1,:,:,:].reshape((i+1,464,1,82,82)))
    distance_matrix[:i,:,:i,:] = distance
    generator = generator_matrix_distance_follow(fc_train)
    parallel = Parallel(n_jobs=4,require='sharedmem',max_nbytes=1e5)
    parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
    np.save(file = "./DistanceMatrices/deep_propofol/distance_matrix_{}_sessions_structure.npy".format(i+1),arr = distance_matrix)

Début de la session 1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

## Light-sevoflurane

In [6]:
dfc_train_light_sevoflurane = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "light-sevoflurane")].reshape(-1,n_wins,82,82)
distance_matrix = np.zeros((1,464,1,464)) 
fc_train = torch.from_numpy(dfc_train_light_sevoflurane[0:1,:,:,:].reshape((1,464,1,82,82)))
real_size = min(11, 82, 82) # window should be atleast 11x11 

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

generator = generator_matrix_distance(fc_train)
parallel = Parallel(n_jobs=3,require='sharedmem',max_nbytes=1e5)
parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
np.save(file = "./DistanceMatrices/light_sevoflurane/distance_matrix_1_sessions_structure.npy",arr = distance_matrix)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [8]:
n=7
dfc_train_light_sevoflurane = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "light-sevoflurane")].reshape(-1,n_wins,82,82)

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

for i in range(1,n):
    print("Début de la session {}".format(i))
    distance_matrix = np.zeros((i+1,464,i+1,464)) 
    distance = np.load("./DistanceMatrices/light_sevoflurane/distance_matrix_{}_sessions_structure.npy".format(i))
    fc_train = torch.from_numpy(dfc_train_light_sevoflurane[0:i+1,:,:,:].reshape((i+1,464,1,82,82)))
    distance_matrix[:i,:,:i,:] = distance
    generator = generator_matrix_distance_follow(fc_train)
    parallel = Parallel(n_jobs=4,require='sharedmem',max_nbytes=1e5)
    parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
    np.save(file = "./DistanceMatrices/light_sevoflurane/distance_matrix_{}_sessions_structure.npy".format(i+1),arr = distance_matrix)

Début de la session 1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

KeyboardInterrupt: 

## Light-propofol

In [9]:
dfc_train_light_propofol = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "light-propofol")].reshape(-1,n_wins,82,82)
distance_matrix = np.zeros((1,464,1,464)) 
fc_train = torch.from_numpy(dfc_train_light_propofol[0:1,:,:,:].reshape((1,464,1,82,82)))
real_size = min(11, 82, 82) # window should be atleast 11x11 

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

generator = generator_matrix_distance(fc_train)
parallel = Parallel(n_jobs=3,require='sharedmem',max_nbytes=1e5)
parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
np.save(file = "./DistanceMatrices/light_propofol/distance_matrix_1_sessions_structure.npy",arr = distance_matrix)

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [10]:
n=7
dfc_train_light_propofol = dfc[(meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])) & (meta["condition"] == "light-propofol")].reshape(-1,n_wins,82,82)

def costly_function(elem,distance_matrix):
    session1,t1,session2,t2,img1,img2 = elem
    accu = metric(img1,img2)
    distance_matrix[session1,t1,session2,t2] = accu
    distance_matrix[session2,t2,session1,t1] = accu

for i in range(6,n):
    print("Début de la session {}".format(i))
    distance_matrix = np.zeros((i+1,464,i+1,464)) 
    distance = np.load("./DistanceMatrices/light_propofol/distance_matrix_{}_sessions_structure.npy".format(i))
    fc_train = torch.from_numpy(dfc_train_light_propofol[0:i+1,:,:,:].reshape((i+1,464,1,82,82)))
    distance_matrix[:i,:,:i,:] = distance
    generator = generator_matrix_distance_follow(fc_train)
    parallel = Parallel(n_jobs=5,require='sharedmem',max_nbytes=1e5)
    parallel(delayed(costly_function)(elem,distance_matrix) for elem in generator)
    np.save(file = "./DistanceMatrices/light_propofol/distance_matrix_{}_sessions_structure.npy".format(i+1),arr = distance_matrix)

Début de la session 1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

KeyboardInterrupt: 